In [3]:
from sqlite3 import connect
import pandas as pd

con = connect('/Users/mtcd/Library/Mobile Documents/com~apple~CloudDocs/@Pessoal/@Evernote/Estudos/@programming/deepface_analysis_datasets/dataset.db')

query = """
-- MAIN RESEARCH
SELECT eamd.file_name, 
        e.emotion as emotion_returned_from_deepface,
        eamd.equivalent_emotion_from_df as emotion_from_dataset,
        er.total_frames,
        max(CASE WHEN eamd.emotion == "angry" THEN eamd.classification END) as angry,       
        max(CASE WHEN eamd.emotion == "disgust" THEN eamd.classification END) as disgust,
        max(CASE WHEN eamd.emotion == "fear" THEN eamd.classification END) as fear,
        max(CASE WHEN eamd.emotion == "neutral" THEN eamd.classification END) as neutral,
        max(CASE WHEN eamd.emotion == "sad" THEN eamd.classification END) as sad,
        max(CASE WHEN eamd.emotion == "surprise" THEN eamd.classification END) as surprise,
        max(CASE WHEN eamd.emotion == "happy" THEN eamd.classification END) as happy,
        eamd.dataset
FROM emotion_analysed_mead_dataset eamd
INNER JOIN emotion_record er ON er.file_name = eamd.file_name
INNER JOIN emotion e ON e.id = er.emotion_captured
WHERE eamd.file_name NOT LIKE "%ontempt%"
GROUP BY eamd.file_name
"""

df = pd.read_sql(query, con)

In [4]:
def summaryze_result_without_one_emotion(df, emotion_excluded):
    dataset_searched = ['ADFES-BIV', 'MEAD']
    # Line1 --> emotion from dataset
    # Line2 --> emotion analysed from deepface
    # Line3 --> emotion most captured without any emotion
    summary = []

    for dataset_name in dataset_searched:
        data_frame_filtered = df.loc[df['dataset'] == dataset_name]
        qtde_acertos = 0
        for line in data_frame_filtered.values:  
            if line[1] == line[2]:
                qtde_acertos += 1
            elif line[1] == line[3]:
                qtde_acertos += 1
        
        summary.append([dataset_name, len(data_frame_filtered), qtde_acertos])

        print("####")
        print("Emotion excluded: ", emotion_excluded)
        print(f'Dataset: {dataset_name}')
        print("Total files analysed: ", len(data_frame_filtered))
        print(f"Total files correctly analysed without {emotion_excluded} emotion: ", qtde_acertos)

In [5]:
# Code to generate informations excluding emotions to see the impact on the result

df = df.loc[(df['dataset'] == 'MEAD') | (df['dataset'] == 'ADFES-BIV')]
emotions = ['angry', 'disgust', 'fear', 'sad', 'surprise', 'happy', 'neutral']

for j in range(len(emotions)):
    list_of_emotions = []
    for i in range(len(emotions)):
        if emotions[j] != emotions[i]:
            list_of_emotions.append(emotions[i])
        else:
            emotion_excluded = emotions[i]
    emotion_excluded
    
    list_of_emotions
    df[f'prevalence_emotion_without_{emotion_excluded}_emotion'] = df[list_of_emotions].idxmax(axis=1)
    df2 = df.loc[:,['file_name', 'emotion_from_dataset', 'emotion_returned_from_deepface', f'prevalence_emotion_without_{emotion_excluded}_emotion', 'total_frames', emotion_excluded,'dataset' ]]

    # Save XLS file with results
    #file_name = f'resultado_{emotion_excluded}_excluded.xlsx'
    #df2.to_excel(file_name)

    summaryze_result_without_one_emotion(df2, emotion_excluded)


####
Emotion excluded:  angry
Dataset: ADFES-BIV
Total files analysed:  504
Total files correctly analysed without angry emotion:  149
####
Emotion excluded:  angry
Dataset: MEAD
Total files analysed:  4543
Total files correctly analysed without angry emotion:  1028
####
Emotion excluded:  disgust
Dataset: ADFES-BIV
Total files analysed:  504
Total files correctly analysed without disgust emotion:  149
####
Emotion excluded:  disgust
Dataset: MEAD
Total files analysed:  4543
Total files correctly analysed without disgust emotion:  985
####
Emotion excluded:  fear
Dataset: ADFES-BIV
Total files analysed:  504
Total files correctly analysed without fear emotion:  149
####
Emotion excluded:  fear
Dataset: MEAD
Total files analysed:  4543
Total files correctly analysed without fear emotion:  995
####
Emotion excluded:  sad
Dataset: ADFES-BIV
Total files analysed:  504
Total files correctly analysed without sad emotion:  149
####
Emotion excluded:  sad
Dataset: MEAD
Total files analysed:  4

In [6]:
df = df.loc[(df['dataset'] == 'MEAD') | (df['dataset'] == 'ADFES-BIV')]

# Visualizar o valor máximo das colunas discriminadas
#df['prevalence_emotion_without_neutral'] = df.loc[:,['angry', 'disgust', 'fear', 'sad', 'surprise', 'happy']].max(1)

# Visualizar o nome da coluna com o maior valor
df['prevalence_emotion_without_neutral'] = df[['angry', 'disgust', 'fear', 'sad', 'surprise', 'happy']].idxmax(axis=1)

df2 = df.loc[:,['file_name', 'emotion_from_dataset', 'emotion_returned_from_deepface', 'prevalence_emotion_without_neutral', 'total_frames', 'neutral','dataset' ]]
df2

,file_name,emotion_from_dataset,emotion_returned_from_deepface,prevalence_emotion_without_neutral,total_frames,neutral,dataset
0,@datasets_adfes_train_angry_F01angryhighlast,angry,sad,sad,1,NaN,ADFES-BIV
1,@datasets_adfes_train_angry_F01angrylowlast,angry,neutral,NaN,1,1.0,ADFES-BIV
2,@datasets_adfes_train_angry_F01angrymedlast,angry,neutral,NaN,1,1.0,ADFES-BIV
3,@datasets_adfes_train_angry_F02angryhighlast,angry,neutral,NaN,1,1.0,ADFES-BIV
4,@datasets_adfes_train_angry_F02angrylowlast,angry,neutral,NaN,1,1.0,ADFES-BIV
...,...,...,...,...,...,...,...
5099,W024_video_front_surprised_level_3_051,surprise,happy,happy,182,74.0,MEAD
5100,W024_video_front_surprised_level_3_052,surprise,happy,happy,103,23.0,MEAD
5101,W024_video_front_surprised_level_3_053,surprise,happy,happy,173,47.0,MEAD
5102,W024_video_front_surprised_level_3_057,surprise,happy,happy,113,20.0,MEAD
